# Autoencoders for images

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle

## Define autoencoder

In [24]:
def create_layer(x, input_size, output_size, name):
    """Helper function for creating single layer"""
    with tf.name_scope(name):
        weights = tf.Variable(tf.random_normal([input_size, output_size], dtype=tf.float32), name='weigths')
        biases = tf.Variable(tf.zeros([output_size]), name='biases')
        return tf.matmul(x, weights) + biases

    
class Autoencoder(object):
    
    def __init__(self, input_size, hidden_size, epoch=250, learning_rate=0.001):
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.x = tf.placeholder(dtype=tf.float32, shape=[None, input_size])
        self.encoder = tf.nn.tanh(create_layer(self.x, input_size, 1, 'encoder'))
        self.decoder = create_layer(self.encoder, 1, input_size, 'decoder')
        self.loss = tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(self.x, self.decoder))))
        self.train_op = tf.train.RMSPropOptimizer(learning_rate).minimize(self.loss)
        self.saver = tf.train.Saver()
        
    def train(self, data, batch_size=10):
        num_samples = len(data)
        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.epoch):
                arr = np.arange(num_samples)
                np.random.shuffle(arr)
                for j in range(num_samples // batch_size):
                    batch_data = data[j*batch_size:(j+1)*batch_size]
                    l, _ = sess.run([self.loss, self.train_op], feed_dict={self.x:batch_data})
                if i % (self.epoch // 10) == 0:
                    print('epoch {}, loss={}'.format(i, l))     
                    self.saver.save(sess, '../models/autoencoder2.ckpt')
            self.saver.save(sess, '../models/autoencoder2.ckpt')
            
    def test(self, data):
        with tf.Session() as sess:
            self.saver.restore(sess, '../models/autoencoder2.ckpt')
            hidden, output = sess.run([self.encoder, self.decoder], feed_dict={self.x: data})
        print('input', data)
        print('compressed', hidden)
        print('output', output)
        return output

Helper functions for reading serialized images

In [29]:
def unpickle(file):
    fo = open(file, 'rb')
    data = pickle.load(fo, encoding='latin1')
    fo.close
    return data


def grayscale(a):
    return a.reshape(a.shape[0], 3, 32, 32).mean(1).reshape(a.shape[0], -1)


def load_images(path):
    names = unpickle(path + 'batches.meta')['label_names']
    data, labels = [], []
    for i in range(1, 6):
        filename = path + 'data_batch_' + str(i)
        batch_data = unpickle(filename)
        if len(data) > 0:
            data = np.vstack([data, batch_data['data']])
            labels = np.hstack([labels, batch_data['labels']])
        else:
            data = batch_data['data']
            labels = batch_data['labels']
    return grayscale(data), labels

## Train autoencoder

Use only horses to speed up training

In [32]:
data, labels = load_images('../datasets/cifar-10-batches-py/')
x = np.matrix(data)
y = np.matrix(labels)

horse_indices = np.where(y==7)[0]
horse_x = x[horse_indices]
print(np.shape(horse_x))

input_size = np.shape(horse_x)[1]
hidden_size = 100
ae = Autoencoder(input_size, hidden_size, 500)
ae.train(horse_x)

(5000, 1024)
epoch 0, loss=110.14291381835938
epoch 100, loss=33.42665100097656
epoch 200, loss=4.5679450035095215
epoch 300, loss=0.00025717710377648473
epoch 400, loss=0.0002567209303379059
epoch 500, loss=0.0002571705263108015
epoch 600, loss=0.0002571738441474736
epoch 700, loss=0.0002567688643466681
epoch 800, loss=0.00025710713816806674
epoch 900, loss=0.00025718522374518216
